In [2]:
import cv2
import numpy as np
import math
import time as time
import random as rand

In [158]:
class Bird:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.gravity = 0.981
        self.jump_amount = 10 * self.gravity
        self.acc = 0
        self.vel = 0

    def draw(self, canvas):
        cv2.putText(canvas, '({}, {})'.format(self.x, self.y), (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0))
        cv2.circle(canvas, (self.x, self.y), radius=3, color=(255, 0, 0), thickness=-1)
        # draw circle on canvas to represent bird location
        
    def update_motion(self):
        self.acc = self.gravity
        self.vel += self.acc
        self.y = int(self.y + self.vel)
        
    def jump(self):
        self.vel = -self.jump_amount

In [194]:
class Obstacle:
    def __init__(self, x, width, height_top, height_bot):
        self.x = x
        self.width = width
        self.height_top = height_top
        self.height_bot = height_bot
    
    def draw(self, canvas):
        height, width = canvas.shape
        cv2.rectangle(canvas, (self.x, 0), (self.x + self.width, self.height_top), (255, 0, 0), -1)
        cv2.rectangle(canvas, (self.x, height - self.height_bot), (self.x + self.width, height), (255, 0, 0), -1)
        # draw obstacle on canvas 

    def update_motion(self):
        self.x -= 2

In [195]:
class Game:
    def __init__(self):
        # set up canvas
        self.canvas_width = 500
        self.canvas_height = 500
        self.canvas = np.zeros((self.canvas_height, self.canvas_width))
        
        # set up bird
        self.bird_init_x = 50
        self.bird_init_y = self.canvas_height // 2
        self.bird = Bird(self.bird_init_x, self.bird_init_y)
        
        # setup obstacle
        self.min_obs_height = 10
        self.obs_offset = 100
        self.obs_gap = 100
        self.obs_width = 50
        self.heights = [(113, 287), (386, 14), (188, 212), (362, 38), (151, 249), (322, 78), (360, 40), (120, 280), (227, 173), (362, 38), (345, 55), (240, 160), (259, 141), (118, 282), (140, 260), (202, 198), (319, 81), (189, 211), (333, 67), (367, 33), (97, 303), (389, 11), (287, 113), (48, 352), (204, 196), (280, 120), (46, 354), (100, 300), (58, 342), (350, 50), (104, 296), (264, 136), (349, 51), (129, 271), (11, 389), (158, 242), (35, 365), (124, 276), (279, 121), (257, 143), (154, 246), (263, 137), (364, 36), (160, 240), (170, 230), (246, 154), (187, 213), (351, 49), (82, 318), (171, 229), (263, 137), (355, 45), (29, 371), (293, 107), (288, 112), (129, 271), (270, 130), (382, 18), (244, 156), (93, 307), (260, 140), (29, 371), (183, 217), (162, 238), (99, 301), (182, 218), (355, 45), (311, 89), (130, 270), (226, 174), (241, 159), (273, 127), (267, 133), (249, 151), (384, 16), (185, 215), (52, 348), (262, 138), (72, 328), (344, 56), (45, 355), (246, 154), (133, 267), (296, 104), (80, 320), (232, 168), (331, 69), (253, 147), (269, 131), (289, 111), (305, 95), (280, 120), (378, 22), (358, 42), (121, 279), (337, 63), (134, 266), (14, 386), (24, 376), (274, 126), (113, 287), (364, 36), (72, 328), (351, 49), (168, 232), (326, 74), (34, 366), (30, 370), (122, 278), (19, 381), (88, 312), (220, 180), (188, 212), (305, 95), (225, 175), (298, 102), (61, 339), (89, 311), (263, 137), (319, 81), (203, 197), (362, 38), (158, 242), (207, 193), (350, 50), (111, 289), (158, 242), (250, 150), (315, 85), (244, 156), (198, 202), (105, 295), (26, 374), (305, 95), (120, 280), (255, 145), (323, 77), (43, 357), (236, 164), (213, 187), (126, 274), (23, 377), (372, 28), (226, 174), (173, 227), (390, 10), (178, 222), (25, 375), (220, 180), (134, 266), (223, 177), (73, 327), (228, 172), (170, 230), (363, 37), (227, 173), (337, 63), (55, 345), (142, 258), (226, 174), (349, 51), (313, 87), (65, 335), (222, 178), (126, 274), (52, 348), (95, 305), (335, 65), (366, 34), (334, 66), (142, 258), (143, 257), (385, 15), (148, 252), (147, 253), (323, 77), (160, 240), (338, 62), (29, 371), (134, 266), (82, 318), (314, 86), (195, 205), (139, 261), (251, 149), (366, 34), (143, 257), (321, 79), (162, 238), (352, 48), (67, 333), (151, 249), (331, 69), (158, 242), (237, 163), (295, 105), (329, 71), (314, 86), (268, 132), (58, 342)]
        self.stored_obstacles = [Obstacle(self.canvas_width - self.obs_width, self.obs_width, height_top, height_bot) for (height_top, height_bot) in self.heights]
        self.obstacles = [self.get_next_obstacle()]
        
#         heights2 = []
#         for i in range(200):
#             height_top = rand.randint(self.min_obs_height, self.canvas_height - self.min_obs_height - self.obs_gap)
#             height_bot = self.canvas_height - height_top - self.obs_gap
#             heights2.append((height_top, height_bot))
#         print(heights2)
        
    def get_next_obstacle(self):
        return self.stored_obstacles.pop(0)
        
    def draw(self):
        self.bird.draw(self.canvas)
        for obs in self.obstacles:
            obs.draw(self.canvas)
        
        # display actual canvas
        cv2.imshow('canvas', self.canvas)
        
    def update(self):
        self.bird.update_motion()
        for obs in self.obstacles:
            obs.update_motion()
        
    def run(self):
        while True:
            self.draw()
            self.update()
            
            # clear canvas
            del self.canvas
            self.canvas = np.zeros((self.canvas_height, self.canvas_width))
            
            k = cv2.waitKey(30) & 0xFF
            if k == ord('q'):
                break
            elif k == ord(' '): # jump
                self.bird.jump()
                
        cv2.destroyAllWindows()

In [196]:
game = Game()
game.run()

In [94]:
cv2.destroyAllWindows()

[(422, 28), (352, 98), (394, 56), (321, 129), (312, 138), (106, 344), (149, 301), (277, 173), (221, 229), (314, 136), (140, 310), (165, 285), (86, 364), (181, 269), (154, 296), (82, 368), (424, 26), (226, 224), (143, 307), (175, 275), (282, 168), (404, 46), (360, 90), (373, 77), (432, 18), (65, 385), (68, 382), (387, 63), (159, 291), (369, 81), (70, 380), (260, 190), (149, 301), (387, 63), (26, 424), (429, 21), (406, 44), (59, 391), (205, 245), (292, 158), (192, 258), (438, 12), (407, 43), (153, 297), (237, 213), (362, 88), (331, 119), (170, 280), (342, 108), (16, 434), (60, 390), (12, 438), (71, 379), (130, 320), (45, 405), (265, 185), (337, 113), (181, 269), (246, 204), (281, 169), (161, 289), (116, 334), (139, 311), (234, 216), (185, 265), (182, 268), (310, 140), (80, 370), (390, 60), (57, 393), (440, 10), (299, 151), (223, 227), (318, 132), (297, 153), (172, 278), (418, 32), (291, 159), (209, 241), (201, 249), (301, 149), (250, 200), (262, 188), (162, 288), (283, 167), (12, 438), (